# <font color="#418FDE" size="6.5" uppercase>**Project Design**</font>

>Last update: 20260131.
    
By the end of this Lecture, you will be able to:
- Define a concrete project goal, dataset choice, and evaluation metrics for a PyTorch capstone. 
- Select an appropriate model architecture and training strategy aligned with project constraints. 
- Plan an experiment schedule that includes baselines, ablations, and potential optimizations. 


## **1. Scoping Your Project**

### **1.1. Selecting a Project Domain**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_01.jpg?v=1769863066" width="250">



>* Choose a domain matching interests, skills, constraints
>* Focus on real users, problems, and clear goals

>* Match domain with its natural data type
>* Ensure tasks and evaluation fit project constraints

>* Match project ambition to time, hardware, experience
>* Right-size the domain to enable focused experimentation



### **1.2. Checking Data Sources**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_02.jpg?v=1769863081" width="250">



>* Inspect datasets carefully for content and labels
>* Ensure data truly supports your project goals

>* Check dataset size, class balance, and coverage
>* Ensure data represents target population; adjust or augment

>* Check legal, technical, and resource data limits
>* Consider privacy, consent, and potential harms



### **1.3. Defining success metrics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_01_03.jpg?v=1769863096" width="250">



>* Turn vague goals into measurable success metrics
>* Pick primary and secondary metrics matching real impact

>* Match metrics to task type and data
>* Choose metrics reflecting real-world costs and benefits

>* Set realistic metric targets using simple baselines
>* Balance performance, speed, size, fairness, and stopping



In [ ]:
#@title Python Code - Defining success metrics

# This script shows simple success metrics selection.
# We use a toy classification example with imbalanced labels.
# Focus is on defining and comparing different evaluation metrics.

# import required standard libraries for numerical work.
import numpy as np

# set deterministic random seed for reproducible results.
np.random.seed(42)

# create small synthetic dataset with binary labels.
true_labels = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# create model A predicted probabilities for positive class.
probs_a = np.array([0.10, 0.05, 0.20, 0.15, 0.30, 0.60, 0.70, 0.80, 0.55, 0.65])

# create model B predicted probabilities for positive class.
probs_b = np.array([0.05, 0.10, 0.15, 0.10, 0.25, 0.90, 0.85, 0.80, 0.70, 0.60])

# validate shapes to avoid broadcasting mistakes.
assert true_labels.shape == probs_a.shape == probs_b.shape

# define helper to convert probabilities to class predictions.
def probs_to_labels(probs, threshold):
    # apply threshold to get binary predictions.
    return (probs >= threshold).astype(int)


# define helper to compute confusion matrix counts.
def confusion_counts(y_true, y_pred):
    # compute true positives, false positives, true negatives, false negatives.
    tp = int(np.sum((y_true == 1) & (y_pred == 1)))
    fp = int(np.sum((y_true == 0) & (y_pred == 1)))
    tn = int(np.sum((y_true == 0) & (y_pred == 0)))
    fn = int(np.sum((y_true == 1) & (y_pred == 0)))
    return tp, fp, tn, fn


# define helper to compute accuracy, precision, recall safely.
def compute_metrics(y_true, y_pred):
    # get confusion matrix components for metrics.
    tp, fp, tn, fn = confusion_counts(y_true, y_pred)
    total = tp + fp + tn + fn

    # compute accuracy with safe division.
    accuracy = (tp + tn) / total if total > 0 else 0.0

    # compute precision with safe division.
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0

    # compute recall with safe division.
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    return accuracy, precision, recall


# choose threshold representing business tolerance for false alarms.
threshold = 0.5

# get hard predictions for both models.
preds_a = probs_to_labels(probs_a, threshold)

# get hard predictions for both models.
preds_b = probs_to_labels(probs_b, threshold)

# compute metrics for model A.
acc_a, prec_a, rec_a = compute_metrics(true_labels, preds_a)

# compute metrics for model B.
acc_b, prec_b, rec_b = compute_metrics(true_labels, preds_b)

# print short explanation of project goal and metric focus.
print("Goal: detect rare positives, prioritize recall over accuracy.")

# print metrics for model A in one concise line.
print("Model A -> accuracy", round(acc_a, 2), "precision", round(prec_a, 2), "recall", round(rec_a, 2))

# print metrics for model B in one concise line.
print("Model B -> accuracy", round(acc_b, 2), "precision", round(prec_b, 2), "recall", round(rec_b, 2))

# decide which model wins under recall focused success metric.
better_model = "A" if rec_a >= rec_b else "B"

# print final success metric decision for this project.
print("Chosen primary metric: recall, selected model", better_model)




## **2. Choosing Model Architectures**

### **2.1. Task Aligned Architectures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_01.jpg?v=1769863158" width="250">



>* Match model type to task and data
>* Use task structure to avoid misaligned architectures

>* Match architecture to output type and granularity
>* Use specialized structures for dense, ranked, paired outputs

>* Match model complexity to task difficulty and data
>* Consider latency, interpretability, robustness, and deployment limits



### **2.2. Reuse or Build Anew**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_02.jpg?v=1769863171" width="250">



>* Choose reuse or scratch based on problem constraints
>* Pretrained suits limited data; scratch suits specialized tasks

>* Reuse stable models to reduce project risk
>* Build custom models for deeper learning and novelty

>* Start from pretrained backbones, then customize key parts
>* Match training plan to architecture choice and rationale



In [ ]:
#@title Python Code - Reuse or Build Anew

# This script compares reusing and building models simply.
# We use TensorFlow to simulate PyTorch style choices.
# Focus on architecture reuse versus building from scratch.

# !pip install tensorflow.

# Import required standard libraries safely.
import os
import random
import numpy as np

# Import TensorFlow and check version.
import tensorflow as tf

# Set deterministic seeds for reproducibility.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)

# Configure TensorFlow random seed deterministically.
tf.random.set_seed(seed_value)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Create a tiny synthetic dataset for binary classification.
num_samples = 200
input_dim = 10

# Generate random features with controlled distribution.
X = np.random.randn(num_samples, input_dim).astype("float32")

# Generate simple labels based on a linear rule.
linear_scores = X.sum(axis=1)
y = (linear_scores > 0).astype("float32")

# Validate dataset shapes before training.
assert X.shape == (num_samples, input_dim)
assert y.shape == (num_samples,)

# Split into small train and validation subsets.
train_size = 160
X_train, X_val = X[:train_size], X[train_size:]

# Split labels into matching subsets.
y_train, y_val = y[:train_size], y[train_size:]

# Define a reusable base model representing pretrained features.
base_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(16, activation="relu"),
])

# Freeze base model to simulate feature extraction reuse.
base_model.trainable = False

# Build a reuse strategy model using frozen base features.
reuse_model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

# Compile reuse model with conservative learning rate.
reuse_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

# Train reuse model briefly with silent verbose setting.
reuse_history = reuse_model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=16,
    verbose=0,
    validation_data=(X_val, y_val),
)

# Evaluate reuse model performance on validation data.
reuse_loss, reuse_acc = reuse_model.evaluate(
    X_val,
    y_val,
    verbose=0,
)

# Build a from scratch model with all layers trainable.
scratch_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

# Compile scratch model with slightly larger learning rate.
scratch_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

# Train scratch model briefly with silent verbose setting.
scratch_history = scratch_model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=16,
    verbose=0,
    validation_data=(X_val, y_val),
)

# Evaluate scratch model performance on validation data.
scratch_loss, scratch_acc = scratch_model.evaluate(
    X_val,
    y_val,
    verbose=0,
)

# Print concise comparison of both strategies.
print("Reuse model validation accuracy:", float(reuse_acc))
print("Scratch model validation accuracy:", float(scratch_acc))
print("Reuse model trainable parameters:", reuse_model.count_params())
print("Scratch model trainable parameters:", scratch_model.count_params())
print("Base model trainable flag:", base_model.trainable)




### **2.3. Computing Budget Tradeoffs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_02_03.jpg?v=1769863233" width="250">



>* Model size must match your compute budget
>* Prefer smaller models to iterate, learn, refine

>* Split compute budget into training, validation, experimentation
>* Prefer smaller pretrained models to enable rich experimentation

>* Stage training and use efficiency techniques thoughtfully
>* Match model and training to hardware, latency needs



## **3. Designing Experiment Baselines**

### **3.1. Defining Strong Baselines**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_01.jpg?v=1769863254" width="250">



>* Strong baselines are simple, honest reference models
>* They reveal if complex methods truly add value

>* Start with simple, task-aware models, then refine
>* Stabilize training, tune key hyperparameters, avoid unfair comparisons

>* Match baselines to real project constraints and tradeoffs
>* Document assumptions, metrics, and improvements over baselines



### **3.2. Designing Ablation Studies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_02.jpg?v=1769863268" width="250">



>* Use ablations to test each design choice
>* Run targeted variants to find truly essential components

>* Change one component per ablation, ask why
>* Keep everything else fixed for fair comparisons

>* Prioritize ablations on costly or uncertain components
>* Use small, targeted runs to gain insights



In [ ]:
#@title Python Code - Designing Ablation Studies

# This script demonstrates simple ablation studies.
# We compare a baseline model and two ablations.
# Focus on planning controlled PyTorch style experiments.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import numpy as np

# Import TensorFlow for quick experiments.
import tensorflow as tf

# Set deterministic random seeds everywhere.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Load MNIST dataset from Keras utilities.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reduce dataset size for faster experiments.
small_train_size = 6000
small_test_size = 1000
x_train = x_train[:small_train_size]
y_train = y_train[:small_train_size]

# Slice test data to keep experiments quick.
x_test = x_test[:small_test_size]
y_test = y_test[:small_test_size]

# Normalize images to zero one range.
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Add channel dimension for convolutional layers.
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Validate shapes before building models.
print("Train shape:", x_train.shape, "Test shape:", x_test.shape)


# Define a function building the baseline model.
def build_baseline_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            16,
            (3, 3),
            activation="relu",
            input_shape=(28, 28, 1),
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


# Define a function building no dropout ablation.
def build_no_dropout_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            16,
            (3, 3),
            activation="relu",
            input_shape=(28, 28, 1),
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


# Define a function building small capacity ablation.
def build_small_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            8,
            (3, 3),
            activation="relu",
            input_shape=(28, 28, 1),
        ),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model


# Define a helper function training and evaluating.
def train_and_evaluate(model_builder, label):
    model = model_builder()
    history = model.fit(
        x_train,
        y_train,
        epochs=2,
        batch_size=64,
        verbose=0,
        validation_split=0.1,
    )

    test_loss, test_acc = model.evaluate(
        x_test,
        y_test,
        verbose=0,
    )

    print(label, "test accuracy:", round(float(test_acc), 4))

    return test_acc


# Run baseline experiment first for comparison.
baseline_acc = train_and_evaluate(build_baseline_model, "Baseline model")

# Run ablation removing dropout regularization.
no_dropout_acc = train_and_evaluate(build_no_dropout_model, "No dropout ablation")

# Run ablation reducing model capacity.
small_model_acc = train_and_evaluate(build_small_model, "Small model ablation")

# Summarize results to support ablation planning.
print("Baseline accuracy:", round(float(baseline_acc), 4))
print("No dropout accuracy:", round(float(no_dropout_acc), 4))
print("Small model accuracy:", round(float(small_model_acc), 4))




### **3.3. Optimization roadmap**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master PyTorch 2.10.0/Module_10/Lecture_A/image_03_03.jpg?v=1769863349" width="250">



>* Plan staged, low-risk to complex improvements
>* Each step has hypothesis, limited changes, criteria

>* Start with cheap, simple changes for gains
>* Delay expensive experiments until basics are optimized

>* Use results to choose next optimization steps
>* Keep a clear experiment log and rationale



# <font color="#418FDE" size="6.5" uppercase>**Project Design**</font>


In this lecture, you learned to:
- Define a concrete project goal, dataset choice, and evaluation metrics for a PyTorch capstone. 
- Select an appropriate model architecture and training strategy aligned with project constraints. 
- Plan an experiment schedule that includes baselines, ablations, and potential optimizations. 

<font color='yellow'>Congratulations on completing this course!</font>